# Search Auto Complete

### N-gram Algorithm
First learn from sample text the frequency of n-word sequence.

In [1]:
from pyspark import SparkContext, SparkConf
sc = SparkContext()

Because pyspark use `\n` as the delimiter when reading files, we need to manually remove all `\n` (or `\r\n` depending on the system) in the input files and replace all `.` as `\n`.
This can be done by running the following code in bash
```
tr -d '\r\n' < *.txt | tr '.' '\n' > combined_input.txt
```

In [10]:
# build n-gram from sample text
no_NGram = 6     # number of n for n gram model, length of sequence

import re
def nGramBuilder(line):
    ans = []
    line = line.strip().lower()
    line = re.sub("[^a-z]", " ", line)
    words = line.strip().split()
    if len(words) < 2:
        return ans
    for i in xrange(len(words)):
        for j in xrange(1, no_NGram):
            if i + j < len(words):
                ans.append((tuple(words[i:i+j+1]), 1))
            else:
                break
    return ans
    
text_RDD = sc.textFile('AutoComplete/bookList/combined_input.txt')
text_RDD.take(10)

nGram_RDD = text_RDD.flatMap(nGramBuilder).reduceByKey(lambda a, b : a+b)
nGram_RDD.take(10)

[((u'minutiae', u'of', u'dress'), 2),
 ((u'david', u'in', u'his'), 1),
 ((u'at', u'the', u'beginning'), 10),
 ((u'meat', u'more'), 1),
 ((u'cupid', u's', u'golden'), 1),
 ((u'dorigen', u'that'), 1),
 ((u'alison', u'and', u'nicholas'), 1),
 ((u'proverb', u'of', u'a', u'shrew'), 1),
 ((u'no', u'remedy', u'in', u'this'), 1),
 ((u'it', u'traces', u'her', u'gradual', u'progress'), 1)]

When we see the first n-1 words, we know the probability of the next word by looking at the n-gram results. Some sequence only appeared a few time so we will discard them. A given input may have many possible predictions, we only want the top ones.

In [21]:
# build language model
threshold = 10      # min frequency to consider in the model
top_k = 10           # no of suggestion to show

def langModelFilter(pair):
    key, value = pair
    return len(key) >= 2 and value > threshold

def nGram2Predict(pair):
    key, value = pair
    start = ' '.join(key[:-1]) + ' '
    predict = key[-1]
    return ( start, (predict, value) )

def selectTop(pair):
    key, values = pair
    freq_dict = {}
    for value in values:
        if value[1] in freq_dict:
            freq_dict[value[1]].append(value[0])
        else:
            freq_dict[value[1]] = [value[0]]
    ans = []
    for i in sorted(freq_dict.keys(), reverse = True):
        ans.extend(freq_dict[i])
        if ans > threshold:
            break
    return (key, ans)

lang_model_RDD = nGram_RDD.filter(langModelFilter).map(nGram2Predict).groupByKey().map(selectTop)
lang_model_RDD.takeSample(withReplacement = False, num = 50)

[(u'with ', [u'the']),
 (u'the poet ', [u's']),
 (u'never ', [u'was']),
 (u'unto ', [u'the']),
 (u'sir ', [u'quoth']),
 (u'religious ', [u'and']),
 (u'he was one ', [u'of']),
 (u'valentine ', [u's']),
 (u'as ye ', [u'shall']),
 (u'green ', [u'and']),
 (u'well ', [u'as']),
 (u'gan ', [u'to']),
 (u'words ', [u'of']),
 (u'over ', [u'the']),
 (u'the nun ', [u's']),
 (u'became ', [u'a']),
 (u'up with ', [u'the']),
 (u'large ', [u'and']),
 (u'anon ', [u'and']),
 (u'and in ', [u'the', u'his']),
 (u'up and ', [u'down']),
 (u'as ye shall ', [u'hear']),
 (u'him ', [u'to']),
 (u'and gan ', [u'to']),
 (u'thee ', [u'and']),
 (u'panel ', [u'pictures']),
 (u'beauty ', [u'of']),
 (u'dead ', [u'and']),
 (u'she ', [u'was']),
 (u'may ', [u'be']),
 (u'i say ', [u'that', u'not']),
 (u'general ', [u'bibliography']),
 (u'appearance ', [u'of']),
 (u'sea ', [u'and']),
 (u'same ', [u'time']),
 (u'it has ', [u'been']),
 (u'o ', [u'er']),
 (u'christ ', [u'and']),
 (u'regard ', [u'for']),
 (u'artists ', [u'of']),
